## Jesuit communities in China ("Chrétientés") by Dehergne

Based on Dehergne, J. (1973). Répertoire des Jésuites de Chine de 1552 à 1800. Institutum historicum ; Letouzey & Ané. https://archive.org/details/bhsi37

Complemented with Dehergne, J. (1957). Les Chrétientés de Chine de la Période Ming (1581–1650). Monumenta Serica, 16(1–2), 1–136. https://doi.org/10.1080/02549948.1957.11730960 where more detailed information is provided.

The Répertoire includes two lists of Jesuits residences in China. One reports to 1644 and the second to 1701.

- [Planche: Carte des Chrétientés Chinoises de la fin des Ming (1644)](https://archive.org/details/bhsi37/page/398/mode/1up) and [map](https://archive.org/details/bhsi37/page/n390/mode/1up) Kleiotranscription at [sources/dehergne-locations-1644.cli](../sources/dehergne-locations-1644.cli)
- [VII. Carte des résidences de Chine en l'année 1701](https://archive.org/details/bhsi37/page/n400/mode/1up) and [map](https://archive.org/details/bhsi37/page/n392/mode/1up) kleio transcription at [sources/dehergne-locations-1701.cli](../sources/dehergne-locations-1701.cli).

### Overview
This notebook produces tables with the names of communties, their wkidata code, and comments related to
modern chinese spelling of the place name. 

The notebook also uses records related to place names in individual missionary entries in the "Repertoire"
to list all the people present in a community at a given point in time.

### Output

**Excel files produced in this notebook:**
- [residences-1644.xlsx](../inferences/residences-1644.xlsx)
- [residences-1701.xlsx](../inferences/residences-1701.xlsx)
- [residences-1644-1701.xlsx](../inferences/residences-1644-1701.xlsx)
- [residences-names-1644-1701.xlsx](../inferences/residences-names-1644-1701.xlsx)
- [residences-1644-1701-no-wikidata.xlsx](../inferences/residences-1644-1701-no-wikidata.xlsx)

**Map produced:**
- [residences_1644_map.html](../inferences/residences_1644_map.html)


### Setup

In [57]:
import timelink

print("Timelink version:", timelink.__version__)

tlnb = timelink.notebooks.TimelinkNotebook()
# tlnb.print_info()

Timelink version: 1.1.26


## List of residences in 1644

Loop through the list of residences in 1644 in file `dehergne-locations-1644.cli` and 
build a dataframe with the administrative structure and the wikidate of the locations. 


In [60]:
import pandas as pd
from sqlalchemy import select
from dehergne_util import get_wikidata_id

geo1, geo2, geo3 = tlnb.db.get_model(['geo1','geo2','geo3'])
stmt = select(geo1).where(geo1.inside == 'deh-chre-1644')

place_list = []

with tlnb.db.session() as session:
    result = session.execute(stmt).fetchall()
    for province, in result:
        comment, wikidata = get_wikidata_id(province, if_missing='No wikidata')
        # print(province.id, province.name, wikidata, comment)
        place_list.append({'province': province.name,
                           'id': province.id,
                           'level': 'province',
                           'name': province.name,
                           'name_original': province.with_extra_info().name,
                           'province_wikidata_id': wikidata,
                           'wikidata_id': wikidata,
                           'comment': comment})
        fous = session.execute(select(geo2).where(geo2.inside == province.id)).fetchall()
        province_wikdata = wikidata
        for fou, in fous:

            comment, wikidata = get_wikidata_id(fou, if_missing='No wikidata')
            # print(' ', fou.name,  wikidata, comment)
            place_list.append({'province': province.name,
                               'province_id': province.id,
                               'province_wikidata_id': province_wikdata,
                               'id': fou.id,
                               'level': 'fou',
                               'fou':fou.name,
                               'name': fou.name,
                               'name_original': fou.with_extra_info().name,
                               'fou_wikidata_id': wikidata,
                               'wikidata_id': wikidata,
                               'comment': comment})
            geo3s = session.execute(select(geo3).where(geo3.inside == fou.id)).fetchall()
            fou_wikidata = wikidata
            for tcheou_hien, in geo3s:
                comment, wikidata = get_wikidata_id(tcheou_hien, if_missing='No wikidata')
                # print('   ', tcheou_hien.name, wikidata,comment,  )
                place_list.append({'province': province.name,
                                    'province_id': province.id,
                                    'province_wikidata_id': province_wikdata,
                                    'fou_id': fou.id,
                                    'fou_wikidata_id': fou_wikidata,
                                    'id': tcheou_hien.id,
                                    'level': 'tcheou-hien',
                                    'fou':fou.name,
                                    'name': tcheou_hien.name,
                                    'name_original': tcheou_hien.with_extra_info().name,
                                    'wikidata_id': wikidata,
                                    'comment': comment})


#### Show one place in kleio (for debugging purposes)


In [61]:
place_kleio_id = "deh-r1644-tsinkiang-tchou"

from timelink.api.models import Geoentity

with tlnb.db.session() as session:
    geo_place = session.get(Geoentity, place_kleio_id)
    print(geo_place.to_kleio(width=60, show_contained=True))



geo3$Tsinkiang#"""Chingchiang, today Jingjiang, 靖江, @wikidata:Q428587, In the original book (Dehergne, 1973), it is written as "Tsinkiang" (without g), which is wrongly spelled, because from both phonetic and geographical points of view, it should be Tsingkiang 靖江. In Dehergne(1973), it is written as Tsingjiang. And in the Chinese translation, it is also recognized as "靖江"."""/geo3
  atr$activa/sim/1635
  atr$geoentity:name@wikidata/"https://www.wikidata.org/wiki/Q428587"#"""Chingchiang, today Jingjiang, 靖江, @wikidata:Q428587, In the original book (Dehergne, 1973), it is written as "Tsinkiang" (without g), which is wrongly spelled, because from both phonetic and geographical points of view, it should be Tsingkiang 靖江. In Dehergne(1973), it is written as Tsingjiang. And in the Chinese translation, it is also recognized as "靖江"."""%Q428587/1644


In [62]:
pd.set_option('display.max_rows', 300)
# create a dataframe from the list
places_1644_df = pd.DataFrame(place_list)
places_1644_df.info()
places_1644_df['year'] = 1644
cols=['year','id','level','province',  'fou','name', 'name_original', 'province_wikidata_id', 'fou_wikidata_id','wikidata_id', 'comment']
places_1644_df[cols].sort_values(by=['province', 'fou', 'name']).head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 215 entries, 0 to 214
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   province              215 non-null    object
 1   id                    215 non-null    object
 2   level                 215 non-null    object
 3   name                  215 non-null    object
 4   name_original         215 non-null    object
 5   province_wikidata_id  215 non-null    object
 6   wikidata_id           215 non-null    object
 7   comment               215 non-null    object
 8   province_id           197 non-null    object
 9   fou                   197 non-null    object
 10  fou_wikidata_id       197 non-null    object
 11  fou_id                135 non-null    object
dtypes: object(12)
memory usage: 20.3+ KB


,year,id,level,province,fou,name,name_original,province_wikidata_id,fou_wikidata_id,wikidata_id,comment
89,1644,deh-r1644-chuchow-tcheou,fou,Anhwei,Chuchow,Chuchow,"Chuchow%Chuchow, today Chuzhou, 滁州, @wikidata:...",Q40956,Q114045,Q114045,"Chuchow, today Chuzhou, 滁州,"
90,1644,deh-r1644-hweichow,fou,Anhwei,Hweichow,Hweichow,"Hweichow#Hoei-tcheou, today Huizhou, 徽州, @wiki...",Q40956,Q4358404,Q4358404,"Hoei-tcheou, today Huizhou, 徽州,"
92,1644,deh-r1644-tungment,tcheou-hien,Anhwei,Hweichow,Tungmen,"Tungmen#""""""(?) In Dehergne(1957, p51): ""Chréti...",Q40956,Q4358404,No wikidata,"(?) In Dehergne(1957, p51): ""Chrétienté au Wuy..."
91,1644,deh-r1644-wuyan-hien,tcheou-hien,Anhwei,Hweichow,Wuyan hien,"Wuyan hien#Ou-yuen, today Wuyuan Xian, 婺源县, @w...",Q40956,Q4358404,Q1357710,"Ou-yuen, today Wuyuan Xian, 婺源县, , Wuyuan hist..."
88,1644,deh-r1644-anhwei,province,Anhwei,NaN,Anhwei,"Anhwei#Anhui, Ngon-hoei, today Anhui, 安徽, @wik...",Q40956,NaN,Q40956,"Anhui, Ngon-hoei, today Anhui, 安徽,"


Export list of residences in 1644 to a file and list of wikidata
ids so they can be used to fetch coordinates and other information.

Ensure that notebook `wikidata-linked-data.ipynb` is run after the next cell to update cache of linked data.


In [63]:
places_1644_df.to_excel('../inferences/residences-1644.xlsx', index=False)
# save to wikidata ids directory
places_1644_df['wikidata_id'].to_csv('../inferences/wikidata-references/residences-1644.csv', index=False)

### Link with wikidata data

Requires that `wikidata-linked-data.ipynb` has been run first to generate the `locations_wikidata_info` file used here.


In [67]:
# load wikidata data
from dehergne_util import locations_wikidata_info_file

# load the locations wikidata info from xlsx file
print("Loading locations wikidata info from", locations_wikidata_info_file)
locations_wikidata_info = pd.read_excel(locations_wikidata_info_file, dtype=str)
locations_wikidata_info.set_index('wikidata_id', inplace=True)

# merge the two dataframes
merged_df = pd.merge(places_1644_df, locations_wikidata_info, on='wikidata_id', how='left')
cols = ['level', 'province', 'fou', 'name', 'wikidata_id', 'coordinates','latitude','longitude','comment', 'province_wikidata_id', 'fou_wikidata_id']
merged_df[cols].sort_values(by=['province', 'fou', 'name']).head(20)

Loading locations wikidata info from ../inferences/wikidata-references/locations_wikidata_info.xlsx


Loading locations wikidata info from ../inferences/wikidata-references/locations_wikidata_info.xlsx


,level,province,fou,name,wikidata_id,coordinates,latitude,longitude,comment,province_wikidata_id,fou_wikidata_id
89,fou,Anhwei,Chuchow,Chuchow,Q114045,"(32.30621, 118.31148)",32.30621,118.31148,"Chuchow, hoje:Chuzhou, 滁州,",Q40956,Q114045
90,fou,Anhwei,Hweichow,Hweichow,Q4358404,"(29.869722222222222, 118.42194444444445)",29.86972222222222,118.4219444444444,"Hoei-tcheou, hoje:Huizhou, 徽州,",Q40956,Q4358404
92,tcheou-hien,Anhwei,Hweichow,Tungmen,No wikidata,NaN,NaN,NaN,,Q40956,Q4358404
91,tcheou-hien,Anhwei,Hweichow,Wuyan hien,Q1357710,"(29.25, 117.85)",29.25,117.85,"Ou-yuen, hoje:Wuyuan Xian, 婺源县, , Wuyuan histo...",Q40956,Q4358404
88,province,Anhwei,NaN,Anhwei,Q40956,"(31.833333333333, 117)",31.833333333333,117,"Anhui, Ngon-hoei, hoje:Anhui, 安徽,",Q40956,NaN
4,fou,Chekiang,Chüchow,Chüchow,Q58235,"(28.95445, 118.8763)",28.95445,118.8763,"K'iu-tcheou, hoje:Quzhou, 衢州, , in the Chinese...",Q16967,Q58235
2,tcheou-hien,Chekiang,Hangchou,Fuyang,Q1011103,"(30.04998, 119.93697)",30.04998,119.93697,"Fou-yang, hoje:Fuyang, 富阳,",Q16967,Q4970
1,fou,Chekiang,Hangchou,Hangchou,Q4970,"(30.25, 120.1675)",30.25,120.1675,"Hang-tcheou, hoje: Hangzhou, 杭州,",Q16967,Q4970
3,tcheou-hien,Chekiang,Hangchou,Jenho,Q9385136,NaN,NaN,NaN,"Jen-houo, hoje: Renhe, 仁和县 (), Historical coun...",Q16967,Q4970
5,fou,Chekiang,Huchow,Huchow,Q42664,"(30.8925, 120.0875)",30.8925,120.0875,"Hou-tcheou, hoje: Huzhou, 湖州,",Q16967,Q42664


Missing coordinates

In [60]:
# list rows where coordinates are not available
missing_coords = merged_df[merged_df['coordinates'].isna()]
print(f"Rows with missing coordinates ({len(missing_coords)}):")
missing_coords[cols].sort_values(by=['province', 'fou', 'name'])


Rows with missing coordinates (34):


,level,province,fou,name,wikidata_id,coordinates,latitude,longitude,comment,province_wikidata_id,fou_wikidata_id
92,tcheou-hien,Anhwei,Hweichow,Tungmen,No wikidata,NaN,NaN,NaN,,Q40956,Q4358404
3,tcheou-hien,Chekiang,Hangchou,Jenho,Q9385136,NaN,NaN,NaN,"Jen-houo, hoje: Renhe, 仁和县 (), Historical coun...",Q16967,Q4970
16,tcheou-hien,Chekiang,Ningpo,Wuking,No wikidata,NaN,NaN,NaN,"Uchim,Ou kin, hoje:? Dehergne(1957) did not gi...",Q16967,Q42780
33,tcheou-hien,Fukien,Changchow,Aupua,Q14420305,NaN,NaN,NaN,"hoje: Houban, 后坂 (), (@geonames:1977135)Au-poa...",Q41705,Q68814
28,tcheou-hien,Fukien,Foochow,Niensien,No wikidata,NaN,NaN,NaN,"Dehergne(1957, p30): ""A côté de Hai keu on c...",Q41705,Q68481
45,tcheou-hien,Fukien,Funing,Tingteo,No wikidata,NaN,NaN,NaN,藤头？顶头？ in the Chinese translation it is recogn...,Q41705,Q241877
66,tcheou-hien,Fukien,Taiwan,Camarri,No wikidata,NaN,NaN,NaN,"hoje:Jinbaoli, 金包里, in the Chinese translation...",Q41705,Q22502
65,tcheou-hien,Fukien,Taiwan,Taparri,Q7420445,NaN,NaN,NaN,"""""""hoje：大包里 in the Chinese translation it is r...",Q41705,Q22502
81,tcheou-hien,Hukwang,Kingchow,Meng kia k'i,No wikidata,NaN,NaN,NaN,,Q1014420,Q14135188
86,tcheou-hien,Hunan,Yungchow,Paishui,No wikidata,NaN,NaN,NaN,,Q45761,Q266014


### Scan for coordinates

If there is no corresponding wikidata entry, or the wikidata entry does not have coordinates, but a geo location was obtained it is introduced in the comments of the transcription and can be retrived here.

In [68]:
import re
from dehergne_util import extract_coordinates

# set the coordinates for the places that have coordinates in the comment column
# interate rows with iterrows
for index, row in merged_df[merged_df['coordinates'].isna()].iterrows():
    # print()
    # print("Looking for coordinates of:", row['id'], row['name'])
    # if row['comment'] is not NaN and contains "coordinates", "latitude", or "longitude"
    if not re.search(r'coordinates:|latitude:|longitude:', row['comment'], flags=re.IGNORECASE):
        # print("No coordinates found in comment for row:", index, row['name'])
        continue
    coords = extract_coordinates(row['comment'])
    if coords:
        print()
        print(row['comment'])
        print("Found coordinates:", coords)
        merged_df.at[index, 'coordinates'] = coords
        merged_df.at[index, 'latitude'] = coords[0]
        merged_df.at[index, 'longitude'] = coords[1]



Looking for coordinates of: 3 Jenho
Jen-houo, hoje: Renhe, 仁和县 (), Historical county name, coordinates: 30.448897N, 120.307504E
Found coordinates: (30.448897, 120.307504)

Looking for coordinates of: 16 Wuking
No coordinates found in comment for row: 16 Wuking

Looking for coordinates of: 28 Niensien
No coordinates found in comment for row: 28 Niensien

Looking for coordinates of: 33 Aupua
No coordinates found in comment for row: 33 Aupua

Looking for coordinates of: 45 Tingteo
No coordinates found in comment for row: 45 Tingteo

Looking for coordinates of: 65 Taparri
"""hoje：大包里 in the Chinese translation it is recognized as 塔巴里,in Dehergne(1957) it is noted as 大包里, which cannot be found in the map. Dehergne(1957) puts it as "Peninsule de Masu, à 1 lieue nord-ouest de San Salvador (社寮)"
                today's Santissima Trinidad ， with Masu coordinates:25.201218764354735N, 121.68618375811317E"""
Found coordinates: (25.201218764354735, 121.68618375811317)

Looking for coordinates of:

### List places with no coordinates

These are the ones that where not geo referenced so far.

In [62]:
# List rows in merged_df where the 'coordinates' column is NaN
missing_coords_df = merged_df[merged_df['coordinates'].isna()]
missing_coords_df

,province,level,name,province_wikidata_id,wikidata_id,comment,fou,fou_wikidata_id,year,chinese_label,...,portuguese_description,coordinates,latitude,longitude,administrative_entity_id,administrative_entity_label_en,administrative_entity_label_zh,country_id,country_label,label
16,Chekiang,tcheou-hien,Wuking,Q16967,No wikidata,"Uchim,Ou kin, hoje:? Dehergne(1957) did not gi...",Ningpo,Q42780,1644,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,Fukien,tcheou-hien,Niensien,Q41705,No wikidata,"Dehergne(1957, p30): ""A côté de Hai keu on c...",Foochow,Q68481,1644,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,Fukien,tcheou-hien,Aupua,Q41705,Q14420305,"hoje: Houban, 后坂 (), (@geonames:1977135)Au-poa...",Changchow,Q68814,1644,后坂村,...,NaN,NaN,NaN,NaN,Q10868117,Shangyong Town,上涌镇,Q148,People's Republic of China,Houban / 后坂村
45,Fukien,tcheou-hien,Tingteo,Q41705,No wikidata,藤头？顶头？ in the Chinese translation it is recogn...,Funing,Q241877,1644,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81,Hukwang,tcheou-hien,Meng kia k'i,Q1014420,No wikidata,,Kingchow,Q14135188,1644,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86,Hunan,tcheou-hien,Paishui,Q45761,No wikidata,,Yungchow,Q266014,1644,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87,Kiangnan,province,Kiangnan,Q11133842,Q11133842,"Jiangnan?hoje:Jiangnan, 江南, , former province ...",NaN,NaN,1644,江南省,...,NaN,NaN,NaN,NaN,Q8733,Qing dynasty,清朝,Q8733,Qing dynasty,Jiangnan / 江南省
92,Anhwei,tcheou-hien,Tungmen,Q40956,No wikidata,,Hweichow,Q4358404,1644,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,Kiangsu,tcheou-hien,Kaowang,Q16963,Q13721719,"hoje: Gaowang, 高旺, , in the Chinese translatio...",Nanking,Q16666,1644,高旺社区,...,NaN,NaN,NaN,NaN,Q11134699,Jiangpu Subdistrict,江浦街道,Q148,People's Republic of China,/ 高旺社区
102,Kiangsu,tcheou-hien,Wutsin,Q16963,No wikidata,?,Changchow,Q57970,1644,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Map the structure of residences linking places to their enclosing administrative units.

In [63]:
%pip install plotly

3367.06s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


3367.06s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


.bash_profile RUN!
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [70]:
# loop over the merged dataframe and print the information

locations = []

for index, row in merged_df.sort_values(['province', 'fou', 'name']).iterrows():
    print(f" id: {row['id']} Level: {row['level']} Province: {row['province']} {row['province_wikidata_id']}, Fou: {row['fou']} {row['fou_wikidata_id']}, Tcheou/Hien: {row['name']}")
    lat = row['latitude']
    lon = row['longitude']
    place_type = row['type'] if 'type' in row else ''
    label = (
        (f"    {row['name']}") +
        (f" {row['comment']}" if row['comment'] else '') +
        (f" {row['english_label']}" if row['english_label'] else '') +
        (f"({row['english_description']})" if row['english_description'] else '') +
        (f" {row['chinese_label']}" if row['chinese_label'] else '') +
        (f"({row['chinese_description']})" if row['chinese_description'] else '')
    )
    print(label)
    level = row['level']
    if level == 'province':
        print(f"    Province: {row['province']} (Wikidata ID: {row['province_wikidata_id']})")
    elif level == 'fou':
        print(f"    Fou: {row['fou']} (Wikidata ID: {row['fou_wikidata_id']})")
    elif level == 'tcheou_hien':
        print(f"    Tcheou/Hien: {row['name']} (Wikidata ID: {row['wikidata_id']})")

 (Level: fou) Province: Anhwei Q40956, Fou: Chuchow Q114045, Tcheou/Hien: Chuchow
Chuchow Chuchow, hoje:Chuzhou, 滁州,  Chuzhou(prefecture-level city in Anhui, China) 滁州市(中国安徽省的地级市)
  Fou: Chuchow (Wikidata ID: Q114045)
 (Level: fou) Province: Anhwei Q40956, Fou: Hweichow Q4358404, Tcheou/Hien: Hweichow
Hweichow Hoei-tcheou, hoje:Huizhou, 徽州,  Huizhou(region in Anhui and Jiangxi, China) 徽州(nan)
  Fou: Hweichow (Wikidata ID: Q4358404)
 (Level: tcheou-hien) Province: Anhwei Q40956, Fou: Hweichow Q4358404, Tcheou/Hien: Tungmen
Tungmen nan(nan) nan(nan)
 (Level: tcheou-hien) Province: Anhwei Q40956, Fou: Hweichow Q4358404, Tcheou/Hien: Wuyan hien
Wuyan hien Ou-yuen, hoje:Wuyuan Xian, 婺源县, , Wuyuan historically was in Anhui province, but now it is in the present-day Jiangxi province Wuyuan County(county in Shangrao, Jiangxi, China) 婺源县(中国江西省上饶市下辖县)
 (Level: province) Province: Anhwei Q40956, Fou: nan nan, Tcheou/Hien: Anhwei
Anhwei Anhui, Ngon-hoei, hoje:Anhui, 安徽,  Anhui(province of China) 安

In [73]:
from textwrap import wrap
import plotly.graph_objects as go

# generate a dictionary for quick lookup with id as key
locations_wikidata_dict = {row['id']: row for index, row in merged_df.iterrows()}

# Define marker styles for each level
level_styles = {
    'province': {'color': 'red', 'size': 12},
    'fou': {'color': 'blue', 'size': 8},
    'tcheou-hien': {'color': 'green', 'size': 5}
}

fig = go.Figure()


for level, style in level_styles.items():
    fig.add_trace(go.Scattermap(
        lon=[None],
        lat=[None],
        mode='markers',
        marker=dict(color=style['color'], size=style['size']),
        name=level
    ))
# Add lines for 'fou' and 'tcheou_hien'
for _, row in merged_df.iterrows():
    level = row.get('level')
    if level == 'fou':
        id_origin = row.get('province_id', None)
        id_destination = row.get('id', None)
        lat_origin = locations_wikidata_dict.get(id_origin, {}).get('latitude', None)
        lon_origin = locations_wikidata_dict.get(id_origin, {}).get('longitude', None)
        lat_destination = locations_wikidata_dict.get(id_destination, {}).get('latitude', None)
        lon_destination = locations_wikidata_dict.get(id_destination, {}).get('longitude', None)
        if lat_origin and lon_origin and lat_destination and lon_destination:
            fig.add_trace(go.Scattermap(
                lon=[float(lon_origin), float(lon_destination)],
                lat=[float(lat_origin), float(lat_destination)],
                mode='lines',
                line=dict(width=1, color='red'),
                showlegend=False
            ))
    elif level == 'tcheou-hien':
        id_origin = row.get('fou_id', None)
        id_destination = row.get('id', None)
        lat_origin = locations_wikidata_dict.get(id_origin, {}).get('latitude', None)
        lon_origin = locations_wikidata_dict.get(id_origin, {}).get('longitude', None)
        lat_destination = locations_wikidata_dict.get(id_destination, {}).get('latitude', None)
        lon_destination = locations_wikidata_dict.get(id_destination, {}).get('longitude', None)
        if id_destination == 'Q7420445' or row['name'] == "Camarri": # special case for Taparri
            print(f"Debugging: {id_origin} -> {id_destination}")
            pass
        if lat_origin and lon_origin and lat_destination and lon_destination:
            fig.add_trace(go.Scattermap(
                lon=[float(lon_origin), float(lon_destination)],
                lat=[float(lat_origin), float(lat_destination)],
                mode='lines',
                line=dict(width=1, color='blue'),
                showlegend=False
            ))
# Add Markers for each location
for index, row in merged_df.iterrows():
    lat = row.get('latitude')
    lon = row.get('longitude')
    level = row.get('level')
    name = row.get('name')
    english_description = row.get('english_description', '')
    chinese_description = row.get('chinese_description', '')
    wikidata_id = row.get('wikidata_id', '')
    coordinates = row.get('coordinates', '')
    if name == "Camarri":
        print(f"Debugging: {name} with wikidata_id {wikidata_id}")

    if wikidata_id != 'No wikidata':
        english_label = row.get('english_label', '')
        chinese_label = row.get('chinese_label', '')
        portuguese_label = row.get('portuguese_label', '')
        english_description = row.get('english_description', '')
        chinese_description = row.get('chinese_description', '')
        portuguese_description = row.get('portuguese_description', '')
        wikidata_label = (
                         f"en:{english_label} ({english_description})<br>"
                         f"zh:{chinese_label} ({chinese_description})<br>"
                         f"pt:{portuguese_label} ({portuguese_description})"
        )
    else:
        wikidata_label = "No wikidata"
        english_label = ''
        chinese_label = ''
        portuguese_label = ''
        english_description = ''
        chinese_description = ''
        portuguese_description = ''

    name_original = row.get('name_original', '')
    comment = "<br>".join(wrap(name_original, width=40))
    hover_text = f"name: {name}<br>original: {comment}<br><br>wikidata ({wikidata_id})<br>{wikidata_label} <br>coordinates: {coordinates}"


    if level == 'tcheou-hien':
        pass
    if pd.notnull(lat) and pd.notnull(lon) and level in level_styles:
        style = level_styles[level]
        fig.add_trace(go.Scattermap(
            lon=[float(lon)],
            lat=[float(lat)],
            mode='markers+text',
            marker=dict(color=style['color'], size=style['size']),
            text=[name],
            textposition='top center',
            hovertext=[hover_text],
            hoverinfo='text',
            hoverlabel=dict(bgcolor='white', font_size=12),
            showlegend=False
        ))

# fig.update_layout(mapbox_style="open-street-map")
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_traces(textfont=dict(size=6))
fig.update_layout({
    "map": {"center": {"lat": 28.0,
                       "lon": 120.0},
            "zoom": 4,
            "style": "carto-positron",
            }
    },
    title= "Residences 1644",
    annotations=[dict(
        x=0,
        y=-0.1,
        xref="paper",
        yref="paper",
        text="<a href='https://archive.org/details/bhsi37/page/n390/mode/1up'>Source</a>",
        showarrow=False,
        font=dict(size=10)
    )],
    autosize=True,
    )

fig.show(config={"responsive": True})
fig.write_html("../inferences/residences_1644_map.html",
               config={"responsive": True},
               include_plotlyjs=True,
               auto_open=False)


Debugging: Q22502 -> Q7420445


Debugging: Q22502 -> Q7420445


### Wrapping Hover Text
Plotly does not automatically wrap long hover labels. To wrap text, insert HTML `<br>` tags or use Python's `textwrap`:

```python
from textwrap import wrap
raw = f"{name}: {comment}"
# wrap at 30 characters and join with <br>
wrapped = "<br>".join(wrap(raw, width=30))
fig.add_trace(go.Scattermap(
    lon=[lon], lat=[lat], mode='markers',
    hovertext=[wrapped], hoverinfo='text',
    marker=dict(color='blue', size=8)
))
```
This forces line breaks in your hover label at the desired width.